In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']


In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_vmp_meat_deli_store_sku 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH 
store_mn AS (
    SELECT *EXCEPT(rn) FROM (
        SELECT 
            store_id,
            type,
            ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_dim.vmp_meat_deli_store
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.vmp_meat_deli_store)
    ) a WHERE rn = 1
),
product_mn AS (
    SELECT *EXCEPT(rn) FROM (
        SELECT 
            product_id,
            dvt base_uom,
            tray,
            type_n1,
            type_n2,
            type_n3,
            type_n4,
            type_n5,
            type_n6,
            ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_dim.vmp_meat_deli_dmmh
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.vmp_meat_deli_dmmh)
    ) a WHERE rn = 1
),
store_mb AS (
    SELECT *EXCEPT(rn) FROM (
        SELECT 
            store_id,
            type,
            ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_meat_deli_store
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_meat_deli_store)
    ) a WHERE rn = 1
),
product_mb AS (
    SELECT *EXCEPT(rn) FROM (
        SELECT 
            product_id,
            dvt base_uom,
            tray,
            type_n1,
            type_n2,
            type_n3,
            type_n4,
            type_n5,
            type_n6,
            ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_meat_deli_dmmh
        WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_dim.mb_vmp_meat_deli_dmmh)
    ) a WHERE rn = 1
)
SELECT store_id, product_id, type_n1 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N1'
UNION ALL
SELECT store_id, product_id, type_n2 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N2'
UNION ALL
SELECT store_id, product_id, type_n3 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N3'
UNION ALL
SELECT store_id, product_id, type_n4 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N4'
UNION ALL
SELECT store_id, product_id, type_n5 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N5'
UNION ALL
SELECT store_id, product_id, type_n6 value, type FROM store_mn, product_mn WHERE store_mn.type = 'N6'

UNION ALL
SELECT store_id, product_id, type_n1 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N1'
UNION ALL
SELECT store_id, product_id, type_n2 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N2'
UNION ALL
SELECT store_id, product_id, type_n3 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N3'
UNION ALL
SELECT store_id, product_id, type_n4 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N4'
UNION ALL
SELECT store_id, product_id, type_n5 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N5'
UNION ALL
SELECT store_id, product_id, type_n6 value, type FROM store_mb, product_mb WHERE store_mb.type = 'N6'
;
""")